# 競馬予測モデルの包括的改善

このノートブックでは、以下の改善を実装します：
1. 競馬ドメイン知識を活かした特徴量エンジニアリング
2. TimeSeriesSplitによる適切な交差検証
3. ベースラインモデルとの比較
4. Optunaによるハイパーパラメータ最適化
5. SHAP値によるモデル解釈性分析

In [18]:
# 必要なライブラリのインポート
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, precision_recall_curve, f1_score
import optuna
import shap
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# 日本語フォント設定
plt.rcParams['font.sans-serif'] = ['Hiragino Sans', 'Yu Gothic', 'Meirio', 'Takao', 'IPAexGothic', 'IPAPGothic', 'VL PGothic', 'Noto Sans CJK JP']
plt.rcParams['axes.unicode_minus'] = False

## 1. データの読み込みとデータ品質チェック

In [19]:
# データ読み込み関数
def load_race_data():
    """エンコード済みデータを読み込む"""
    # 可能なファイルパスのパターン
    possible_paths = [
        'encoded/encoded_data.csv',
        'encoded/2022_2023encoded_data.csv',
        'encoded/2022encoded_data.csv',
        'encoded/2023encoded_data.csv'
    ]
    
    for path in possible_paths:
        try:
            df = pd.read_csv(path)
            print(f"データを読み込みました: {path}")
            print(f"データサイズ: {df.shape}")
            return df
        except FileNotFoundError:
            continue
    
    raise FileNotFoundError("エンコード済みデータが見つかりません")

# データ読み込み
df = load_race_data()

# データの基本情報を確認
print("\n=== データの基本情報 ===")
print(f"総レコード数: {len(df):,}")
print(f"カラム数: {len(df.columns)}")
print(f"\n欠損値の割合:")
missing_ratio = df.isnull().sum() / len(df) * 100
print(missing_ratio[missing_ratio > 0].sort_values(ascending=False))

データを読み込みました: encoded/2022_2023encoded_data.csv
データサイズ: (75369, 138)

=== データの基本情報 ===
総レコード数: 75,369
カラム数: 138

欠損値の割合:
日付差4       72.168929
日付5        72.168929
日付5_sin    72.168929
日付5_cos    72.168929
日付4        63.412013
日付4_cos    63.412013
日付4_sin    63.412013
日付差3       63.412013
日付3        52.339821
日付差2       52.339821
日付3_cos    52.339821
日付3_sin    52.339821
通過順        47.256830
日付2_sin    38.428266
日付2_cos    38.428266
日付差1       38.428266
日付2        38.428266
日付1_sin    21.056402
日付1        21.056402
日付差        21.056402
日付1_cos    21.056402
dtype: float64


In [20]:
# 日付列の処理と確認
if '日付' in df.columns:
    df['日付'] = pd.to_datetime(df['日付'])
    df = df.sort_values('日付').reset_index(drop=True)
    
    print("\n=== 時系列データの確認 ===")
    print(f"データ期間: {df['日付'].min()} ~ {df['日付'].max()}")
    print(f"\n年別レコード数:")
    df['year'] = df['日付'].dt.year
    print(df['year'].value_counts().sort_index())


=== 時系列データの確認 ===
データ期間: 1970-01-01 00:00:00.000000035 ~ 1970-01-01 00:00:00.000000753

年別レコード数:
year
1970    75369
Name: count, dtype: int64


## 2. 競馬ドメイン知識を活かした特徴量エンジニアリング

In [21]:
def create_domain_features(df):
    """
    競馬のドメイン知識を活かした特徴量を作成
    """
    df_features = df.copy()
    
    print("=== 特徴量エンジニアリング開始 ===")
    
    # 1. 馬の能力指標
    # 前走からの間隔（休養期間）
    if '前走からの間隔' in df_features.columns:
        # 適度な休養（3-5週）フラグ
        df_features['適度な休養'] = df_features['前走からの間隔'].apply(
            lambda x: 1 if 21 <= x <= 35 else 0
        )
        # 長期休養明け（12週以上）フラグ
        df_features['長期休養明け'] = df_features['前走からの間隔'].apply(
            lambda x: 1 if x >= 84 else 0
        )
    
    # 2. 過去成績の集約特徴量
    # 前走着順グループ（勝ち馬、連対、着内、着外）
    if '前走着順' in df_features.columns:
        df_features['前走勝利'] = (df_features['前走着順'] == 1).astype(int)
        df_features['前走連対'] = (df_features['前走着順'] <= 2).astype(int)
        df_features['前走着内'] = (df_features['前走着順'] <= 3).astype(int)
    
    # 3. 距離適性
    if '距離' in df_features.columns:
        # 距離カテゴリ（短距離、マイル、中距離、長距離）
        df_features['短距離'] = (df_features['距離'] <= 1400).astype(int)
        df_features['マイル'] = ((df_features['距離'] > 1400) & (df_features['距離'] <= 1800)).astype(int)
        df_features['中距離'] = ((df_features['距離'] > 1800) & (df_features['距離'] <= 2400)).astype(int)
        df_features['長距離'] = (df_features['距離'] > 2400).astype(int)
    
    # 4. 馬場状態の影響
    if '馬場' in df_features.columns:
        # 馬場状態を数値化（良=0, 稍重=1, 重=2, 不良=3）
        baba_map = {'良': 0, '稍重': 1, '重': 2, '不良': 3}
        df_features['馬場状態数値'] = df_features['馬場'].map(baba_map)
        # 重馬場フラグ
        df_features['重馬場'] = df_features['馬場状態数値'].apply(lambda x: 1 if x >= 2 else 0)
    
    # 5. 斤量の影響
    if '斤量' in df_features.columns and '性別' in df_features.columns:
        # 性別ごとの標準斤量からの差
        df_features['牡馬'] = (df_features['性別'] == '牡').astype(int)
        df_features['牝馬'] = (df_features['性別'] == '牝').astype(int)
        # 斤量負担（牡馬57kg、牝馬55kgを基準）
        df_features['斤量負担'] = df_features.apply(
            lambda row: row['斤量'] - 57 if row['牡馬'] else row['斤量'] - 55,
            axis=1
        )
    
    # 6. 枠順の影響
    if '枠番' in df_features.columns and '頭数' in df_features.columns:
        # 内枠（1-3枠）、中枠（4-6枠）、外枠（7-8枠）
        df_features['内枠'] = (df_features['枠番'] <= 3).astype(int)
        df_features['外枠'] = (df_features['枠番'] >= 7).astype(int)
        # 相対的な枠位置（0-1に正規化）
        df_features['相対枠位置'] = df_features['枠番'] / df_features['頭数']
    
    # 7. 季節・時期の影響
    if '日付' in df_features.columns:
        df_features['月'] = df_features['日付'].dt.month
        # 季節（春：3-5月、夏：6-8月、秋：9-11月、冬：12-2月）
        df_features['春'] = df_features['月'].apply(lambda x: 1 if 3 <= x <= 5 else 0)
        df_features['夏'] = df_features['月'].apply(lambda x: 1 if 6 <= x <= 8 else 0)
        df_features['秋'] = df_features['月'].apply(lambda x: 1 if 9 <= x <= 11 else 0)
        df_features['冬'] = df_features['月'].apply(lambda x: 1 if x == 12 or x <= 2 else 0)
    
    # 8. 人気と実力の乖離
    if '単勝' in df_features.columns and '人気' in df_features.columns:
        # 低人気高配当馬の可能性（人気順位 / オッズのログ）
        df_features['人気オッズ乖離'] = df_features['人気'] / (np.log1p(df_features['単勝']) + 1)
    
    # 9. コース特性
    if 'コース' in df_features.columns:
        # 芝・ダート
        df_features['芝'] = df_features['コース'].str.contains('芝').astype(int)
        df_features['ダート'] = df_features['コース'].str.contains('ダ').astype(int)
        # 右回り・左回り（競馬場による）
        if '競馬場' in df_features.columns:
            left_courses = ['東京', '中京', '新潟']
            df_features['左回り'] = df_features['競馬場'].apply(
                lambda x: 1 if any(course in str(x) for course in left_courses) else 0
            )
    
    # 10. 騎手・調教師の影響
    if '騎手' in df_features.columns:
        # 騎手の勝率（簡易版：上位騎手フラグ）
        top_jockeys = df_features[df_features['着順'] == 1]['騎手'].value_counts().head(20).index
        df_features['上位騎手'] = df_features['騎手'].isin(top_jockeys).astype(int)
    
    print(f"\n作成した特徴量数: {len(df_features.columns) - len(df.columns)}")
    
    return df_features

# 特徴量エンジニアリング実行
df_features = create_domain_features(df)

=== 特徴量エンジニアリング開始 ===

作成した特徴量数: 12


In [22]:
# ターゲット変数の作成
df_features['target'] = (df_features['着順'] <= 3).astype(int)

# 特徴量の選択
exclude_cols = ['着順', 'target', 'オッズ', '人気', '上がり', '走破時間', '通過順', 
                '日付', 'year', '月', 'race_id', '馬番']
feature_cols = [col for col in df_features.columns if col not in exclude_cols]

print(f"\n使用する特徴量数: {len(feature_cols)}")
print(f"ターゲット分布:")
print(df_features['target'].value_counts())
print(f"正例（3着以内）の割合: {df_features['target'].mean():.2%}")


使用する特徴量数: 140
ターゲット分布:
target
0    58716
1    16653
Name: count, dtype: int64
正例（3着以内）の割合: 22.10%


## 3. TimeSeriesSplitによる交差検証の実装

In [23]:
# 時系列交差検証の設定
tscv = TimeSeriesSplit(n_splits=5)

# データを時系列順にソート
df_features = df_features.sort_values('日付').reset_index(drop=True)

# 交差検証用のデータ準備
X = df_features[feature_cols]
y = df_features['target']

print("=== 時系列交差検証の分割 ===")
for i, (train_idx, valid_idx) in enumerate(tscv.split(X)):
    train_dates = df_features.iloc[train_idx]['日付']
    valid_dates = df_features.iloc[valid_idx]['日付']
    print(f"\nFold {i+1}:")
    print(f"  訓練: {train_dates.min().date()} ~ {train_dates.max().date()} ({len(train_idx):,}件)")
    print(f"  検証: {valid_dates.min().date()} ~ {valid_dates.max().date()} ({len(valid_idx):,}件)")

=== 時系列交差検証の分割 ===

Fold 1:
  訓練: 1970-01-01 ~ 1970-01-01 (12,564件)
  検証: 1970-01-01 ~ 1970-01-01 (12,561件)

Fold 2:
  訓練: 1970-01-01 ~ 1970-01-01 (25,125件)
  検証: 1970-01-01 ~ 1970-01-01 (12,561件)

Fold 3:
  訓練: 1970-01-01 ~ 1970-01-01 (37,686件)
  検証: 1970-01-01 ~ 1970-01-01 (12,561件)

Fold 4:
  訓練: 1970-01-01 ~ 1970-01-01 (50,247件)
  検証: 1970-01-01 ~ 1970-01-01 (12,561件)

Fold 5:
  訓練: 1970-01-01 ~ 1970-01-01 (62,808件)
  検証: 1970-01-01 ~ 1970-01-01 (12,561件)


## 4. ベースラインモデル（ロジスティック回帰）の実装

In [24]:
# ベースラインモデルの評価
baseline_scores = []
scaler = StandardScaler()

print("=== ベースラインモデル（ロジスティック回帰）の評価 ===")

for fold, (train_idx, valid_idx) in enumerate(tscv.split(X)):
    X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
    y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]
    
    # 欠損値の処理
    X_train = X_train.fillna(X_train.mean())
    X_valid = X_valid.fillna(X_train.mean())  # 訓練データの平均で埋める
    
    # 標準化
    X_train_scaled = scaler.fit_transform(X_train)
    X_valid_scaled = scaler.transform(X_valid)
    
    # クラス重みの計算
    class_weight = {0: 1, 1: (y_train == 0).sum() / (y_train == 1).sum()}
    
    # モデル学習
    lr_model = LogisticRegression(class_weight=class_weight, max_iter=1000, random_state=42)
    lr_model.fit(X_train_scaled, y_train)
    
    # 予測と評価
    y_pred = lr_model.predict_proba(X_valid_scaled)[:, 1]
    auc_score = roc_auc_score(y_valid, y_pred)
    baseline_scores.append(auc_score)
    
    print(f"Fold {fold+1} AUC: {auc_score:.4f}")

print(f"\nベースライン平均AUC: {np.mean(baseline_scores):.4f} ± {np.std(baseline_scores):.4f}")

=== ベースラインモデル（ロジスティック回帰）の評価 ===


ValueError: Input X contains NaN.
LogisticRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

## 5. LightGBMモデルの交差検証評価

In [ ]:
# LightGBMモデルの評価
lgb_scores = []
lgb_models = []

print("\n=== LightGBMモデルの評価 ===")

for fold, (train_idx, valid_idx) in enumerate(tscv.split(X)):
    X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
    y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]
    
    # 欠損値の処理
    X_train = X_train.fillna(X_train.mean())
    X_valid = X_valid.fillna(X_train.mean())
    
    # クラス重みの計算
    scale_pos_weight = (y_train == 0).sum() / (y_train == 1).sum()
    
    # 基本パラメータ
    params = {
        'objective': 'binary',
        'metric': 'binary_logloss',
        'boosting_type': 'gbdt',
        'scale_pos_weight': scale_pos_weight,
        'random_state': 42,
        'verbosity': -1,
        'num_leaves': 31,
        'learning_rate': 0.05,
        'feature_fraction': 0.9,
        'bagging_fraction': 0.8,
        'bagging_freq': 5,
        'min_child_samples': 20,
        'n_estimators': 300
    }
    
    # モデル学習
    model = lgb.LGBMClassifier(**params)
    model.fit(X_train, y_train, 
             eval_set=[(X_valid, y_valid)],
             callbacks=[lgb.early_stopping(50), lgb.log_evaluation(0)])
    
    # 予測と評価
    y_pred = model.predict_proba(X_valid)[:, 1]
    auc_score = roc_auc_score(y_valid, y_pred)
    lgb_scores.append(auc_score)
    lgb_models.append(model)
    
    print(f"Fold {fold+1} AUC: {auc_score:.4f}")

print(f"\nLightGBM平均AUC: {np.mean(lgb_scores):.4f} ± {np.std(lgb_scores):.4f}")
print(f"\n改善率: {(np.mean(lgb_scores) - np.mean(baseline_scores)) / np.mean(baseline_scores) * 100:.1f}%")

## 6. Optunaによるハイパーパラメータ最適化

In [ ]:
# Optuna最適化の実装
def objective(trial):
    """Optunaの目的関数"""
    # ハイパーパラメータの探索範囲
    params = {
        'objective': 'binary',
        'metric': 'binary_logloss',
        'boosting_type': 'gbdt',
        'verbosity': -1,
        'random_state': 42,
        'num_leaves': trial.suggest_int('num_leaves', 10, 100),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.5, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),
        'min_child_samples': trial.suggest_int('min_child_samples', 10, 100),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 1.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 1.0),
    }
    
    # 交差検証でのスコア計算
    cv_scores = []
    
    for train_idx, valid_idx in tscv.split(X):
        X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
        y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]
        
        # 欠損値処理
        X_train = X_train.fillna(X_train.mean())
        X_valid = X_valid.fillna(X_train.mean())
        
        # クラス重み
        scale_pos_weight = (y_train == 0).sum() / (y_train == 1).sum()
        params['scale_pos_weight'] = scale_pos_weight
        
        # モデル学習
        model = lgb.LGBMClassifier(**params, n_estimators=100)
        model.fit(X_train, y_train,
                 eval_set=[(X_valid, y_valid)],
                 callbacks=[lgb.early_stopping(30), lgb.log_evaluation(0)])
        
        # 予測と評価
        y_pred = model.predict_proba(X_valid)[:, 1]
        cv_scores.append(roc_auc_score(y_valid, y_pred))
    
    return np.mean(cv_scores)

# Optuna最適化実行
print("=== Optunaによるハイパーパラメータ最適化 ===")
print("※時間がかかるため、試行回数を20回に制限しています")

study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=42))
study.optimize(objective, n_trials=20, show_progress_bar=True)

print(f"\n最適なAUCスコア: {study.best_value:.4f}")
print("\n最適なパラメータ:")
for key, value in study.best_params.items():
    print(f"  {key}: {value}")

## 7. 最適化されたモデルでの最終評価

In [ ]:
# 最適化されたパラメータでの再学習
optimized_params = {
    'objective': 'binary',
    'metric': 'binary_logloss',
    'boosting_type': 'gbdt',
    'verbosity': -1,
    'random_state': 42,
    'n_estimators': 300,
    **study.best_params
}

# 最終的な時系列分割（最後の20%をテスト用）
split_point = int(len(X) * 0.8)
X_train_final = X.iloc[:split_point]
y_train_final = y.iloc[:split_point]
X_test_final = X.iloc[split_point:]
y_test_final = y.iloc[split_point:]

# 欠損値処理
X_train_final = X_train_final.fillna(X_train_final.mean())
X_test_final = X_test_final.fillna(X_train_final.mean())

# クラス重み
optimized_params['scale_pos_weight'] = (y_train_final == 0).sum() / (y_train_final == 1).sum()

# 最終モデルの学習
final_model = lgb.LGBMClassifier(**optimized_params)
final_model.fit(X_train_final, y_train_final)

# テストセットでの評価
y_pred_final = final_model.predict_proba(X_test_final)[:, 1]
final_auc = roc_auc_score(y_test_final, y_pred_final)

print("=== 最終モデルの評価結果 ===")
print(f"テストAUC: {final_auc:.4f}")

# 最適閾値の探索
precisions, recalls, thresholds = precision_recall_curve(y_test_final, y_pred_final)
f1_scores = 2 * (precisions * recalls) / (precisions + recalls + 1e-10)
best_idx = np.argmax(f1_scores[:-1])
optimal_threshold = thresholds[best_idx]

print(f"最適閾値: {optimal_threshold:.4f}")
print(f"F1スコア: {f1_scores[best_idx]:.4f}")

## 8. SHAP値によるモデル解釈性分析

In [ ]:
# SHAP値の計算
print("=== SHAP値によるモデル解釈 ===")
print("※計算に時間がかかるため、サンプル数を制限しています")

# テストデータから1000件をランダムサンプリング
sample_size = min(1000, len(X_test_final))
sample_idx = np.random.choice(X_test_final.index, size=sample_size, replace=False)
X_sample = X_test_final.loc[sample_idx]

# SHAP値の計算
explainer = shap.TreeExplainer(final_model)
shap_values = explainer.shap_values(X_sample)

# 正例（3着以内）に対するSHAP値を使用
if isinstance(shap_values, list):
    shap_values = shap_values[1]

In [ ]:
# 特徴量重要度のプロット
plt.figure(figsize=(10, 8))
shap.summary_plot(shap_values, X_sample, plot_type="bar", show=False)
plt.title("特徴量重要度（SHAP値）", fontsize=16)
plt.xlabel("平均|SHAP値|")
plt.tight_layout()
plt.show()

In [ ]:
# SHAP Summary Plot
plt.figure(figsize=(10, 8))
shap.summary_plot(shap_values, X_sample, show=False)
plt.title("特徴量の影響度分析", fontsize=16)
plt.xlabel("SHAP値（モデル出力への影響）")
plt.tight_layout()
plt.show()

In [ ]:
# 上位特徴量の詳細分析
feature_importance = pd.DataFrame({
    'feature': X_sample.columns,
    'importance': np.abs(shap_values).mean(axis=0)
}).sort_values('importance', ascending=False)

print("=== 重要な特徴量トップ20 ===")
for i, row in feature_importance.head(20).iterrows():
    print(f"{row['feature']:30} 重要度: {row['importance']:.4f}")

In [ ]:
# 個別予測の説明（Waterfall plot）
# 3着以内になりそうな馬の例
high_prob_idx = np.where(y_pred_final[sample_idx] > 0.5)[0]
if len(high_prob_idx) > 0:
    idx = high_prob_idx[0]
    plt.figure(figsize=(10, 6))
    shap.waterfall_plot(shap.Explanation(values=shap_values[idx], 
                                        base_values=explainer.expected_value[1] if isinstance(explainer.expected_value, list) else explainer.expected_value,
                                        data=X_sample.iloc[idx],
                                        feature_names=X_sample.columns.tolist()),
                       show=False)
    plt.title(f"個別予測の説明（予測確率: {y_pred_final[sample_idx][idx]:.2%}）", fontsize=14)
    plt.tight_layout()
    plt.show()

## 9. モデル比較と改善点のまとめ

In [ ]:
# モデル比較結果のまとめ
print("=== モデル性能比較 ===")
print(f"\n1. ベースライン（ロジスティック回帰）:")
print(f"   平均AUC: {np.mean(baseline_scores):.4f} ± {np.std(baseline_scores):.4f}")

print(f"\n2. LightGBM（初期パラメータ）:")
print(f"   平均AUC: {np.mean(lgb_scores):.4f} ± {np.std(lgb_scores):.4f}")
print(f"   改善率: +{(np.mean(lgb_scores) - np.mean(baseline_scores)) / np.mean(baseline_scores) * 100:.1f}%")

print(f"\n3. LightGBM（最適化後）:")
print(f"   テストAUC: {final_auc:.4f}")
print(f"   ベースラインからの改善: +{(final_auc - np.mean(baseline_scores)) / np.mean(baseline_scores) * 100:.1f}%")

print("\n=== 実装した改善点 ===")
print("✅ 1. 競馬ドメイン知識を活かした特徴量エンジニアリング")
print("   - 休養期間、距離適性、馬場状態、斤量負担など")
print("   - 作成した新規特徴量: {}個".format(len(df_features.columns) - len(df.columns)))

print("\n✅ 2. TimeSeriesSplitによる適切な交差検証")
print("   - 5分割の時系列交差検証を実装")
print("   - データリークを防ぐ正しい評価")

print("\n✅ 3. ベースラインモデルとの比較")
print("   - ロジスティック回帰でベースライン性能を確立")
print("   - LightGBMの優位性を定量的に確認")

print("\n✅ 4. Optunaによるハイパーパラメータ最適化")
print("   - 20試行でパラメータを自動最適化")
print("   - 交差検証スコアを目的関数として使用")

print("\n✅ 5. SHAP値によるモデル解釈性分析")
print("   - 特徴量の重要度と影響を可視化")
print("   - 個別予測の説明も可能に")

In [ ]:
# 今後の改善提案
print("\n=== 今後の改善提案 ===")
print("\n1. さらなる特徴量エンジニアリング:")
print("   - 馬の血統情報の活用")
print("   - 調教データの組み込み")
print("   - レース間の相互作用特徴量")

print("\n2. アンサンブル手法の導入:")
print("   - 複数モデルのスタッキング")
print("   - 異なるアルゴリズムの組み合わせ")

print("\n3. より詳細な評価:")
print("   - 回収率シミュレーション")
print("   - リスク調整後リターンの計算")
print("   - ケリー基準による最適賭け金の算出")

print("\n4. 外部データの活用:")
print("   - 天候データ")
print("   - 競馬場の特性データ")
print("   - 騎手・調教師の詳細成績")